In [1]:
!pip install pyspark psycopg2-binary

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, TimestampType, ShortType
from pyspark.sql.functions import to_timestamp

In [3]:
def init_spark_session():    
    try:
        spark = SparkSession.builder \
            .appName("PostgreSQL") \
            .config("spark.jars.packages", "org.postgresql:postgresql:42.2.20") \
            .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
            .getOrCreate()

        spark.conf.set("spark.hadoop.fs.s3a.access.key", "datalake")
        spark.conf.set("spark.hadoop.fs.s3a.secret.key", "datalake")
        spark.conf.set("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
        spark.conf.set("spark.hadoop.fs.s3a.path.style.access", "true")
        spark.conf.set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
        print("Sessão Spark criada com sucesso!")

        return spark
    except Exception as err:
        print("Something went wrong trying to create or get spark session!")
        print(f"ERROR: {err}")
        return None

In [4]:
spark = init_spark_session()

Sessão Spark criada com sucesso!


In [5]:
df = spark.read.format("delta").load("s3a://raw/position")

In [6]:
from pyspark.sql.functions import to_timestamp, col

df = df \
    .withColumnRenamed("hora", "hora_info")\
    .withColumnRenamed("utc", "data_hora")\
    .withColumn("hora_info", col("hora_info").cast("string")) \
    .withColumn("cd_linha", col("cd_linha").cast("integer")) \
    .withColumn("sentido", col("sentido").cast("short")) \
    .withColumn("quantidade_veiculos", col("quantidade_veiculos").cast("integer")) \
    .withColumn("flag_acessivel", col("flag_acessivel").cast("boolean")) \
    .withColumn("data_hora", to_timestamp("data_hora")) \
    .withColumn("latitude_veiculo", col("latitude_veiculo").cast("string")) \
    .withColumn("longitude_veiculo", col("longitude_veiculo").cast("string"))

jdbc_url = "jdbc:postgresql://db:5432/spacedb"
properties = {
    "user": "spacedb",
    "password": "spacedb",
    "driver": "org.postgresql.Driver"
}

# Escrever dados no PostgreSQL
df.write.jdbc(url=jdbc_url, table="sparcedb.posicao", mode="append", properties=properties)

print("Dados escritos com sucesso na tabela PostgreSQL!")


Dados escritos com sucesso na tabela PostgreSQL!
